In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
from torchvision import models
from skimage import io, color, transform
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Path to the folders containing real and fake images
original_folder_path = "/content/gdrive/MyDrive/dataset/original-cropped-images"
synthetic_folder_path = "/content/gdrive/MyDrive/dataset/synthetic-cropped-images"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
def load_and_preprocess_images(folder,half):
    images = []

    count=0
    if half==1:
        for filename in os.listdir(folder)[:20000]:
            img_path = os.path.join(folder, filename)
            img = io.imread(img_path)
            img_gray = color.rgb2gray(img)
            img_resized = transform.resize(img_gray, (64, 64))  # Resize to a consistent size

            # Convert the NumPy array to a PyTorch tensor
            img_tensor = torch.tensor(img_resized, dtype=torch.float32).to(device)
            images.append(img_tensor)

            count+=1
            print(count)
    if half==2:
        for filename in os.listdir(folder)[20000:]:
            img_path = os.path.join(folder, filename)
            img = io.imread(img_path)
            img_gray = color.rgb2gray(img)
            img_resized = transform.resize(img_gray, (64, 64))  # Resize to a consistent size

            # Convert the NumPy array to a PyTorch tensor
            img_tensor = torch.tensor(img_resized, dtype=torch.float32).to(device)
            images.append(img_tensor)

            count+=1
            print(count)

    return images

In [ ]:
real_images1 = load_and_preprocess_images(original_folder_path,1)

In [ ]:
real_images2 = load_and_preprocess_images(original_folder_path,2)

In [ ]:
fake_images1 = load_and_preprocess_images(synthetic_folder_path,1)

In [ ]:
fake_images2 = load_and_preprocess_images(synthetic_folder_path,2)

In [ ]:
# Create labels (0 for real, 1 for fake)
real_labels_tensor = torch.zeros(len(real_images1)+len(real_images2), dtype=torch.float32).to(device)
fake_labels_tensor = torch.ones(len(fake_images1)+len(fake_images2), dtype=torch.float32).to(device)

In [ ]:
real_images_tensor1 = torch.stack(real_images1)
real_images_tensor2 = torch.stack(real_images2)
fake_images_tensor1 = torch.stack(fake_images1)
fake_images_tensor2 = torch.stack(fake_images2)

In [ ]:
# Combine real and fake data
all_images = torch.cat((real_images_tensor1,real_images_tensor2, fake_images_tensor1,fake_images_tensor2), dim=0)
all_labels = torch.cat((real_labels_tensor, fake_labels_tensor), dim=0)

In [ ]:
all_images_numpy = all_images.cpu().numpy()
all_labels_numpy = all_labels.cpu().numpy()

# Split the dataset into training and testing sets
X_train_numpy, X_test_numpy, y_train_numpy, y_test_numpy = train_test_split(
    all_images_numpy, all_labels_numpy, test_size=0.2, random_state=42
)

In [ ]:
# Convert the NumPy arrays back to PyTorch tensors (and move to GPU if available)
X_train = torch.tensor(X_train_numpy, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test_numpy, dtype=torch.float32).to(device)
train_labels = torch.tensor(y_train_numpy, dtype=torch.float32).to(device)
test_labels = torch.tensor(y_test_numpy, dtype=torch.float32).to(device)

### LBP

In [ ]:
# Parameters for LBP
radius = 1
n_points = 8 * radius

# Extract LBP features using PyTorch and GPU
def extract_lbp_features(images):
    features = []
    for img in images:
        img = img.squeeze(0).cpu().numpy()  # Convert the tensor to NumPy and remove the batch dimension
        lbp_img = local_binary_pattern(img, n_points, radius, method='uniform')
        hist, _ = np.histogram(lbp_img.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-8)
        features.append(hist)
    return features

In [ ]:
lbp_train_features = extract_lbp_features(X_train)
lbp_test_features = extract_lbp_features(X_test)
lbp_train_features = torch.tensor(lbp_train_features, dtype=torch.float32).to(device)
lbp_test_features = torch.tensor(lbp_test_features, dtype=torch.float32).to(device)

### HOG

In [ ]:
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler

In [ ]:
# Function to extract HOG features from a list of images
def extract_hog_features(images):
    hog_features = []
    for image in images:
        hog_feature = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
        hog_features.append(hog_feature)
    return np.array(hog_features)

In [ ]:
# Extract HOG features from training and testing data
hog_train_features = extract_hog_features(X_train_numpy)
hog_test_features = extract_hog_features(X_test_numpy)

In [ ]:
# Standardize the HOG features
scaler = StandardScaler()
hog_train_features = scaler.fit_transform(hog_train_features)
hog_test_features = scaler.transform(hog_test_features)

In [ ]:
# Convert HOG features to PyTorch tensors
hog_train_features = torch.tensor(hog_train_features, dtype=torch.float32).to(device)
hog_test_features = torch.tensor(hog_test_features, dtype=torch.float32).to(device)

### SIFT

In [ ]:
import cv2

In [ ]:
def extract_sift_features(images):
    sift = cv2.SIFT_create()
    keypoints_list = []
    descriptors_list = []

    for image in images:
        # Perform SIFT feature extraction on CPU
        image8bit = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')
        keypoints, descriptors = sift.detectAndCompute(image8bit, None)

        # Move keypoints and descriptors to GPU (if available)
        keypoints = [kp.pt for kp in keypoints]

        keypoints_list.append(keypoints)
        descriptors_list.append(descriptors)

    return keypoints_list, descriptors_list

In [ ]:
train_keypoints_list, train_descriptors_list = extract_sift_features(X_train_numpy)
test_keypoints_list, test_descriptors_list = extract_sift_features(X_test_numpy)

In [ ]:
def keypoints_to_vectors(keypoints_list, descriptors_list):
    sift_vector_length = 128  # SIFT descriptor length

    features_vectors = []

    for descriptors in descriptors_list:
        if descriptors is None:
            # If no keypoints are detected, add zeros as a placeholder
            features_vectors.append(torch.zeros(sift_vector_length).to(device))
        else:
            # Randomly choose one descriptor from multiple descriptors
            random_idx = np.random.randint(0, descriptors.shape[0])
            features_vectors.append(torch.tensor(descriptors[random_idx], dtype=torch.float32).to(device))

    # Convert the list of vectors to a PyTorch tensor
    features_vectors = torch.stack(features_vectors)

    # Move the tensor to the GPU (if available)
    features_vectors = features_vectors.to(device)

    return features_vectors

In [ ]:
train_fv = keypoints_to_vectors(train_keypoints_list, train_descriptors_list)
test_fv = keypoints_to_vectors(test_keypoints_list, test_descriptors_list)

In [ ]:
# Convert the NumPy arrays back to PyTorch tensors (and move to GPU if available)
sift_train_features = torch.tensor(train_fv, dtype=torch.float32).to(device)
sift_test_features = torch.tensor(test_fv, dtype=torch.float32).to(device)

### Model

In [ ]:
class LbpModel(nn.Module):
    def __init__(self, input_size):
        super(LbpModel, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.5)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input if not already flattened
        x = self.fc1(x)
        return x

In [ ]:
class HogModel(nn.Module):
    def __init__(self, input_size):
        super(HogModel, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(input_size, 2048),
            nn.ReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5)
        )
        self.fc4 = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5)
        )
        self.fc5 = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5)
        )
        self.fc6 = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5)
        )
        self.fc7 = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.5)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input if not already flattened
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        x = self.fc6(x)
        x = self.fc7(x)
        return x

In [ ]:
class SiftModel(nn.Module):
    def __init__(self, input_size):
        super(SiftModel, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5)
        )
        self.fc3 = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5)
        )
        self.fc4 = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.5)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input if not already flattened
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        return x

In [ ]:
lbp_model=LbpModel(lbp_train_features.shape[1])
lbp_model.to(device)

hog_model=HogModel(hog_train_features.shape[1])
hog_model.to(device)

sift_model=SiftModel(sift_train_features.shape[1])
sift_model.to(device)

In [ ]:
class CombinedModel(nn.Module):
    def __init__(self,lbp_model,hog_model,sift_model):
        super(CombinedModel,self).__init__()
        self.lbp_model=lbp_model
        self.hog_model=hog_model
        self.sift_model=sift_model
        self.fc1=nn.Sequential(
            nn.Linear(96, 48),
            nn.ReLU(),
            nn.BatchNorm1d(48),
            nn.Dropout(0.5)
        )
        self.fc2=nn.Sequential(
            nn.Linear(48, 24),
            nn.ReLU(),
            nn.BatchNorm1d(24),
            nn.Dropout(0.5)
        )
        self.fc3=nn.Sequential(
            nn.Linear(24, 1),
            nn.Sigmoid()
        )

    def forward(self,lbp_features,hog_features,sift_features):
        lbp_output=self.lbp_model(lbp_features)
        hog_output=self.hog_model(hog_features)
        sift_output=self.sift_model(sift_features)

        combined_output=torch.cat((lbp_output,hog_output,sift_output),dim=1)

        x=self.fc1(combined_output)
        x=self.fc2(x)
        x=self.fc3(x)

        return x

In [ ]:
combined_model=CombinedModel(lbp_model,hog_model,sift_model)
combined_model.to(device)

In [ ]:
# Define the loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(combined_model.parameters(), lr=0.01)  # Adam optimizer with a learning rate of 0.001

In [ ]:
# Define the number of training epochs
num_epochs=250
batch_size=32

In [ ]:
lbp_train_dataset=TensorDataset(lbp_train_features,train_labels)
lbp_train_loader=DataLoader(lbp_train_dataset,batch_size,shuffle=True)
lbp_test_dataset=TensorDataset(lbp_test_features,test_labels)
lbp_test_loader=DataLoader(lbp_test_dataset,batch_size,shuffle=False)

In [ ]:
hog_train_dataset=TensorDataset(hog_train_features,train_labels)
hog_train_loader=DataLoader(hog_train_dataset,batch_size,shuffle=True)
hog_test_dataset=TensorDataset(hog_test_features,test_labels)
hog_test_loader=DataLoader(hog_test_dataset,batch_size,shuffle=False)

In [ ]:
sift_train_dataset=TensorDataset(sift_train_features,train_labels)
sift_train_loader=DataLoader(sift_train_dataset,batch_size,shuffle=True)
sift_test_dataset=TensorDataset(sift_test_features,test_labels)
sift_test_loader=DataLoader(sift_test_dataset,batch_size,shuffle=False)

In [ ]:
# training
for epoch in range(num_epochs):
    combined_model.train()
    total_loss=0.0

    for (lbp_batch_data,lbp_labels),(hog_batch_data,hog_labels),(sift_batch_data,sift_labels) in zip(lbp_train_loader,hog_train_loader,sift_train_loader):
        optimizer.zero_grad()
        lbp_batch_data,hog_batch_data,sift_batch_data=lbp_batch_data.to(device),hog_batch_data.to(device),sift_batch_data.to(device)

        output=combined_model(lbp_batch_data,hog_batch_data,sift_batch_data)
        loss=criterion(output,hog_labels.view(-1,1).float().to(device))
        loss.backward()
        optimizer.step()

        total_loss+=loss.item()

    avg_train_loss = total_loss / len(lbp_train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Training Loss: {avg_train_loss:.4f}")

In [ ]:
# testing
combined_model.eval()

correct = 0
total = 0

with torch.no_grad():
    for (lbp_batch_data,lbp_labels),(hog_batch_data,hog_labels),(sift_batch_data,sift_labels) in zip(lbp_test_loader,hog_test_loader,sift_test_loader):
        lbp_batch_data,hog_batch_data,sift_batch_data=lbp_batch_data.to(device),hog_batch_data.to(device),sift_batch_data.to(device)
        output=combined_model(lbp_batch_data,hog_batch_data,sift_batch_data)

        predicted = (output>=0.5).float()
        total+=hog_labels.size(0)

        for i in range(hog_labels.size(0)):
            if predicted[i][0]==hog_labels[i]:
                correct += 1

accuracy=correct/total
print(f"Test Accuracy: {accuracy:.4f}%")